In [1]:
!pip install -q datasets transformers accelerate peft bitsandbytes huggingface_hub evaluate rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import os
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import torch
import numpy as np
import evaluate

file_path = "ai-medical-chatbot.csv"
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "yousefsaeedian/ai-medical-chatbot",
    file_path,
)

df["input_text"] = df.apply(lambda row: f"### Instruction:\n{row['Patient']}", axis=1)
df["target_text"] = df.apply(lambda row: f"{row['Doctor']}", axis=1)
df = df[:40000]

dataset = Dataset.from_pandas(df[["input_text", "target_text"]])
dataset = dataset.train_test_split(test_size=0.2)

2025-05-12 17:35:17.121368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747071317.320381      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747071317.385142      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_19/1040173014.py:21: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


In [ ]:

model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


num_encoder_layers_to_freeze = 9

if hasattr(model, 'encoder') and hasattr(model.encoder, 'block'):
    print(f"Freezing the first {num_encoder_layers_to_freeze} encoder layers of {model_name}...")
    for i, block in enumerate(model.encoder.block):
        if i < num_encoder_layers_to_freeze:
            for param in block.parameters():
                param.requires_grad = False
    if hasattr(model, 'shared'):
        for param in model.shared.parameters():
            param.requires_grad = False


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Freezing the first 9 encoder layers of t5-base...


In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    labels = tokenizer(
        text_target=examples["target_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True,
                                remove_columns=["input_text", "target_text"])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
rouge_metric = evaluate.load("rouge")

def compute_metrics_fn(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    try:
        result = rouge_metric.compute(
            predictions=decoded_preds, references=decoded_labels, use_stemmer=True
        )
    except Exception as e:
        print("Error computing ROUGE:", e)
        return {}

    result = {k: v * 100 for k, v in result.items()}
    result["gen_len"] = np.mean([np.count_nonzero(p != tokenizer.pad_token_id) for p in preds])

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_name}-medical-chatbot-finetuned-with-metrics",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    predict_with_generate=True,
    generation_max_length=128,
    report_to="none",
    learning_rate=5e-5,
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_fn
)

print("Starting training...")
train_result = trainer.train()
print("Training finished.")

print("\nEvaluating the best model on the test set...")
metrics = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print(metrics)

print("\nSaving final model and tokenizer...")
trainer.save_model(f"./final_{model_name}_medical_chatbot")
print("Model and tokenizer saved.")

/tmp/ipykernel_19/2299715436.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer( # Changed to Seq2SeqTrainer


Starting training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.767500,2.405568,32.181800,24.464600,31.256600,32.198500,52.237400
2,2.720700,2.405568,32.181800,24.464600,31.256600,32.198500,52.237400
3,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.000000


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3177: RuntimeWarning: invalid value encountered in less
  if operator(metric_value, self.state.best_metric):
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Training finished.

Evaluating the best model on the test set...


{'eval_loss': 2.4055683612823486, 'eval_rouge1': 32.1818, 'eval_rouge2': 24.4646, 'eval_rougeL': 31.2566, 'eval_rougeLsum': 32.1985, 'eval_gen_len': 52.2374, 'eval_runtime': 7202.9785, 'eval_samples_per_second': 1.111, 'eval_steps_per_second': 0.555, 'epoch': 3.0}

Saving final model and tokenizer...
Model and tokenizer saved.
